In [1]:
import sys
#!{sys.executable} -m pip install pytesseract
#!{sys.executable} -m pip install opencv-python

In [6]:
import numpy as np
import cv2
import imutils
from PIL import Image
import pytesseract as tess

In [8]:
# The tesseract library is an optical character recognition (OCR) tool for Python.
# That is, it can recognize and read the text embedded from any image. 
# So we’ll use it for identifying the characters inside the number plate.

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'


img = cv2.imread('C:\\Users\\Likhit\\OneDrive\\Desktop\\ML\\dataset\\1.jpg',cv2.IMREAD_COLOR)

# Resize the image to the required size and then grayscale it.

img = cv2.resize(img, (600,400) )

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

In [9]:
cv2.imshow('Original Image',img)
cv2.waitKey(0)

-1

In [10]:
cv2.imshow('Gray Image',gray)
cv2.waitKey(0)

-1

In [11]:
# using a bilateral filter (Blurring) will remove the unwanted details from an image. 
gray = cv2.bilateralFilter(gray, 13, 15, 15) 
cv2.imshow('After Bilateral Filtering',gray)
cv2.waitKey(0)

-1

In [12]:
# canny edge method for edge detection

edged = cv2.Canny(gray, 30, 200) 
cv2.imshow('Edges ',edged)
cv2.waitKey(0)

-1

In [13]:
# Now we can start looking for contours on our image
# Once the counters have been detected we sort them from big to small and consider only the first 10 results
#ignoring the others. In our image the counter could be anything that has a closed surface but of all
# the obtained results the license plate number will also be there since it is also a closed surface.

contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None

# To filter the license plate image among the obtained results, we will loop though all the results
# and check which has a rectangle shape contour with four sides and closed figure. 
# Since a license plate would definitely be a rectangle four sided figure.

for c in contours:
    
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
 
    if len(approx) == 4:
        screenCnt = approx
        break

if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
     detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)
    


In [14]:
#  So we can proceed with masking the entire picture except for the place where the number plate is

mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv2.bitwise_and(img,img,mask=mask)

cv2.imshow('After Masking ',new_image)
cv2.waitKey(0)

-1

In [16]:

# Number Plate Recognition is to segment the license plate out of the image by cropping it and saving it as a new image
(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]

cv2.imshow('Cropped ',Cropped)
cv2.waitKey(0)


-1

In [17]:
# read the number plate information from the segmented image.
# We will use the pytesseract package to read characters from image,

text = pytesseract.image_to_string(Cropped,config='--psm 11')
#config='--psm 11')

print("License Plate Recognition\n")
print("Detected license plate Number is:",text)


License Plate Recognition

Detected license plate Number is: HR51BV3737



In [18]:
img = cv2.resize(img,(500,300))
Cropped = cv2.resize(Cropped,(400,200))

cv2.imshow('car',img)
cv2.imshow('Cropped',Cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
from IPython.core.display import display, HTML

In [20]:
display(HTML("""<a href="https://vahan.nic.in/nrservices/faces/user/citizen/searchstatus.xhtml">website</a>"""))